In [4]:
module Promise = struct
    type 'a state = Pending | Resolved of 'a | Rejected of exn

    type 'a handler = 'a state -> unit

    type 'a promise = {
      mutable state : 'a state;
      mutable handlers : 'a handler list;
    }

    type 'a resolver = 'a promise

    let write_once p s =
      if p.state = Pending then p.state <- s else invalid_arg "cannot write twice"

    let make () =
      let p = { state = Pending; handlers = [] } in
      (p, p)

    let return x = { state = Resolved x; handlers = [] }

    let state p = p.state

    let resolve_or_reject r s =
      assert (s <> Pending);
      let handlers = r.handlers in
      r.handlers <- [];
      write_once r s;
      List.iter (fun f -> f s) handlers

    let resolve r x = resolve_or_reject r (Resolved x)

    let reject r e = resolve_or_reject r (Rejected e)

    let enqueue handler p = p.handlers <- handler :: p.handlers

    let handler resolver = function
      | Pending -> failwith "violated"
      | Rejected e -> reject resolver e
      | Resolved x -> resolve resolver x

    let handler_of_callback callback resolver : 'a handler = function
      | Pending -> failwith "violated"
      | Rejected e -> reject resolver e
      | Resolved x -> (
          let promise = callback x in
          match promise.state with
          | Rejected e -> reject resolver e
          | Resolved x -> resolve resolver x
          | Pending -> enqueue (handler resolver) promise)

    let ( >>= ) (promise : 'a promise) (callback : 'a -> 'b promise) : 'b promise =
      match promise.state with
      | Resolved x -> callback x
      | Rejected e -> { state = Rejected e; handlers = [] }
      | Pending ->
          let output_promise, output_resolver = make () in
          enqueue (handler_of_callback callback output_resolver) promise;
          output_promise
end

module Promise :
  sig
    type 'a state = Pending | Resolved of 'a | Rejected of exn
    type 'a handler = 'a state -> unit
    type 'a promise = {
      mutable state : 'a state;
      mutable handlers : 'a handler list;
    }
    type 'a resolver = 'a promise
    val write_once : 'a promise -> 'a state -> unit
    val make : unit -> 'a promise * 'a promise
    val return : 'a -> 'a promise
    val state : 'a promise -> 'a state
    val resolve_or_reject : 'a promise -> 'a state -> unit
    val resolve : 'a promise -> 'a -> unit
    val reject : 'a promise -> exn -> unit
    val enqueue : 'a handler -> 'a promise -> unit
    val handler : 'a promise -> 'a state -> unit
    val handler_of_callback : ('a -> 'b promise) -> 'b promise -> 'a handler
    val ( >>= ) : 'a promise -> ('a -> 'b promise) -> 'b promise
  end


In [5]:
Promise.state (Promise.return 42);;

- : int Promise.state = Promise.Resolved 42
